In [ ]:
# !pip3 install pandas numpy matplotlib seaborn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import scipy

In [ ]:
sns.set()

# Preprocess

In [ ]:
%%time
# CPU times: user 28 s, sys: 111 ms, total: 28.1 s
# Wall time: 27.6 s

stdpso = pd.read_csv(f'data/raw/std_pso_{datetime.date.today()}.csv').sort_values('benchmark')

g = sns.FacetGrid(
    data=stdpso,
    col="benchmark", 
    col_wrap=3,
    height=3.5, 
    aspect=1.5,
    sharey=False,
    palette="viridis",
)

g.map_dataframe(
    sns.lineplot,
    x="iter_num",
    y='perc_oob',
#     estimator=None,
#     units='rep_num',
#     ci=None
)
g.add_legend()
g.set_titles(
    col_template="{col_name}", 
#     row_template="{row_name}"
)

g.set_axis_labels("Iteration Number", "Fraction out of bounds")
plt.savefig('../report/figs/standard_pso_oob.pdf')

# Optimise Control Parameters per Benchmark

## Heatmaps of Control Params

In [ ]:
opt_gb['1%'] = opt_gb.groupby('benchmark').gbest_fit.transform(lambda x: x.quantile(0.01))
best_cps = opt_gb[opt_gb.gbest_fit < opt_gb['1%']] \
    .sort_values(['benchmark', 'gbest_fit', 'w', 'c']) \
    .set_index(['benchmark', 'w', 'c'])[['gbest_fit']]
best_cps['gbest_fit'] = np.round(best_cps['gbest_fit'], 3)

with open('../report/table.tex', 'w') as f:
    tex = best_cps.to_latex(
        caption='Optimal control parameter values for all tested functions',
        label="tab_best_control_params"
    )
    f.write(tex.replace(" Function", '') \
          .replace('benchmark', 'Benchmark Function') \
          .replace(' c ', ' $c$ ') \
          .replace(' w ', ' $w$ ') \
          .replace('gbest\\_fit', 'Global Best Fitness')
     )
best_cps

In [ ]:
%%time
# CPU times: user 21.1 s, sys: 2.11 s, total: 23.2 s
# Wall time: 21.8 s

prefs = [
    {
        'key': 'perc_oob',
        'in_parens': 'Fraction out of bounds',
        'save_as': 'opt_heatmaps_perc_oob',
    }, {
        'key': 'gdiversity',
        'in_parens': 'Swarm diversity',
        'save_as': 'opt_heatmaps_gdiversity',
    }, {
        'key': 'gbest_fit',
        'in_parens': 'Global best fitness',
        'save_as': 'opt_heatmaps_gbest_fit',
    },
]
pref = prefs[2]
for pref in prefs:
#     opt = pd.read_csv(f'data/raw/opt_{datetime.date.today()}.csv')
    opt = pd.read_csv(f'data/raw/opt_2022-03-20.csv')
    opt = opt[~opt.benchmark.isin([
        'Deb 3 Function',
        'Generalized Price 2 Function',
        'Mishra 7 Function',
        'Generalized Paviani Function',
    ])]

    opt['c'] = np.round(opt['c1'] + opt['c2'], 4)
    opt['w'] = np.round(opt['w'], 4)

    opt_gb = opt[opt.iter_num == 4900].groupby(['benchmark', 'w', 'c'])[pref['key']] \
                .mean() \
                .reset_index() \
                .sort_values('benchmark')

    def draw_heatmap(*args, **kwargs):
        data = kwargs.pop('data')
        d = data.pivot(columns=args[0], index=args[1], values=args[2])
        d = d.reindex(index=sorted(d.index)[::-1])
        d = d.reindex(columns=sorted(d.columns))
        ax = sns.heatmap(d, **kwargs)
        w_orig = np.arange(0, 1.1, 0.001)
        w = -(w_orig-0.5) + 0.5
        c = (24 - 24 * w * w) / (7 - 5 * w)
        plt.plot(
            (c * 2.5) + 0.5,
            (w_orig * 9.1) + 0.5,
            c='grey'
        )

    g = sns.FacetGrid(opt_gb,
                      col='benchmark', 
                      col_wrap=3, 
                      height=5)

    g.map_dataframe(draw_heatmap, 'c', 'w', pref['key'],
    #                 annot=True,
                    cmap="viridis",
    #                 fmt='.0e',
                    yticklabels=True, 
                    xticklabels=True)

    g.set_titles(
        col_template=f"{{col_name}} ({pref['in_parens']})", 
    )
    g.set_axis_labels("$c=c_1+c_2$", "$w$")

    plt.savefig(f"../report/figs/{pref['save_as']}.pdf")
    plt.show()

## Best fitness value over time

In [ ]:
%%time
# CPU times: user 30min 59s, sys: 7.32 s, total: 31min 6s
# Wall time: 31min 4s

prefs = [
    {
        'key': 'perc_oob',
        'legend': 'c',
        'in_parens': 'Fraction out of bounds',
        'save_as': 'opt_perc_oob',
    }, {
        'key': 'gdiversity',
        'in_parens': 'Swarm diversity',
        'save_as': 'opt_gdiversity',
    }, {
        'key': 'gbest_fit',
        'in_parens': 'Global best fitness',
        'save_as': 'opt_gbest_fitness',
    },
]
pref = prefs[2]
for pref in prefs:
#     opt = pd.read_csv(f'data/raw/opt_{datetime.date.today()}.csv')
    opt = pd.read_csv(f'data/raw/opt_2022-03-20.csv')
    opt = opt[~opt.benchmark.isin([
        'Deb 3 Function',
        'Generalized Price 2 Function',
        'Mishra 7 Function',
        'Generalized Paviani Function',
    ])]

    opt['c'] = np.round(opt['c1'] + opt['c2'], 4)
    opt['w'] = np.round(opt['w'], 4)

    for param in ['c', 'w']:
        opt_ot = opt.groupby(['benchmark', 'w', 'c', 'iter_num']) \
            .gbest_fit.mean() \
            .reset_index() \
            .sort_values('benchmark')

        g = sns.FacetGrid(
            data=opt, 
            col="benchmark",
            hue=param,
            col_wrap=3,
            height=3.5, 
            aspect=1.5,
            sharey=False,
            palette="viridis"
        )
        g.map(
            sns.lineplot, 
            "iter_num", 
            pref['key'],
        #     ci=None
        )
        g.add_legend()
        g.set_titles(
            col_template="{col_name}",
        )
        g.set_axis_labels("Iteration Number", pref['in_parens'])
        plt.savefig(f"../report/figs/{pref['save_as']}_{param}.pdf")
        plt.show()

# Resample from Poli

In [ ]:
%%time
# CPU times: user 12min 50s, sys: 1.75 s, total: 12min 52s
# Wall time: 12min 51s
prefs = [
    {
        'key': 'perc_oob',
        'in_parens': 'Fraction out of bounds',
        'save_as': 'resample_perc_oob_max_stagnent_iters',
    }, {
        'key': 'gdiversity',
        'in_parens': 'Swarm diversity',
        'save_as': 'resample_gdiversity_max_stagnent_iters',
    }, {
        'key': 'gbest_fit',
        'in_parens': 'Global best fitness',
        'save_as': 'resample_gbest_fit_max_stagnent_iters',
    },
]
pref = prefs[0]
for pref in prefs:
#     res = pd.read_csv(f'data/raw/resample_{datetime.date.today()}.csv').sort_values('benchmark')
    res = pd.read_csv(f'data/raw/resample_2022-03-20.csv').sort_values('benchmark')
    res = res[~res.benchmark.isin([
        'Deb 3 Function',
        'Generalized Price 2 Function',
        'Mishra 7 Function',
        'Generalized Paviani Function',
    ])]

    g = sns.FacetGrid(
        data=res, 
        col="benchmark",
        hue='max_stagnent_iters',
        col_wrap=3,
        height=3.5, 
        aspect=1.5,
        sharey=False,
        palette="viridis" # Different colour pallete to distinguish colours better
    )
    g.map(
        sns.lineplot, 
        "iter_num", pref['key'],
    #     ci=None,
    )
    g.add_legend(title='Maximum stagnent iterations')
    g.set_titles(
        col_template=f"{{col_name}}", 
    )
    g.set_axis_labels("Iteration number", pref['in_parens'])

    plt.savefig(f"../report/figs/{pref['save_as']}.pdf")
    plt.show()